<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/10-Adding_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.9.21 openai==1.6.0 tiktoken==0.5.2 chromadb==0.4.21 kaleido==0.2.1 python-multipart==0.0.6 cohere==4.39

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
os.environ["COHERE_API_KEY"] = "<YOUR_COHERE_KEY>"

In [3]:
import nest_asyncio

nest_asyncio.apply()

# Load a Model

In [4]:
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0.9, model="gpt-3.5-turbo", max_tokens=512)

# Create a VectoreStore

In [5]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [6]:
from llama_index.vector_stores import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Load the Dataset (CSV)

## Download

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [7]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

--2024-02-06 19:06:12--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.04s   

2024-02-06 19:06:12 (4.66 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]



## Read File

In [8]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    rows.append( row )

# The number of characters in the dataset.
len( rows )

14

# Convert to Document obj

In [9]:
from llama_index import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}) for row in rows]

# Transforming

In [10]:
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [11]:
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings import OpenAIEmbedding
from llama_index.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3, llm=llm),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=10, llm=llm),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store
)

nodes = pipeline.run(documents=documents, show_progress=True);

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

464
452
457
465
448
468
434
447
455
445
449
455
431
453


100%|██████████| 108/108 [00:48<00:00,  2.24it/s]


Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

In [12]:
len( nodes )

108

In [13]:
!zip -r vectorstore.zip mini-llama-articles

  adding: mini-llama-articles/ (stored 0%)
  adding: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/ (stored 0%)
  adding: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/data_level0.bin (deflated 100%)
  adding: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/header.bin (deflated 61%)
  adding: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/link_lists.bin (stored 0%)
  adding: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/length.bin (deflated 48%)
  adding: mini-llama-articles/chroma.sqlite3 (deflated 65%)


# Load Indexes

In [17]:
!unzip vectorstore.zip

Archive:  vectorstore.zip
   creating: mini-llama-articles/
   creating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/data_level0.bin  
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/header.bin  
 extracting: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/link_lists.bin  
  inflating: mini-llama-articles/a361e92f-9895-41b6-ba72-4ad38e9875bd/length.bin  
  inflating: mini-llama-articles/chroma.sqlite3  


In [18]:
# Create your index
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [19]:
# Create your index
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

# Query Dataset

In [20]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(top_n=2)

In [21]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank]
)

res = query_engine.query("How many parameters LLaMA2 model has?")

In [22]:
res.response

'The LLaMA2 model has four different sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

In [23]:
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 467d71eb-c7c2-4713-8a02-4df1269424ca
Title	 The Generative AI Revolution: Exploring the Current Landscape
Text	 Models Meta AI, formerly known as Facebook Artificial Intelligence Research (FAIR), is an artificial intelligence laboratory that aims to share open-source frameworks, tools, libraries, and models for research exploration and large-scale production deployment. In 2018, they released the open-source PyText, a modeling framework focused on NLP systems. Then, in August 2022, they announced the release of BlenderBot 3, a chatbot designed to improve conversational skills and safety. In November 2022, Meta developed a large language model called Galactica, which assists scientists with tasks such as summarizing academic papers and annotating molecules and proteins. Released in February 2023, LLaMA (Large Language Model Meta AI) is a transformer-based foundational large language model by Meta that ventures into both the AI and academic spaces. The model aims to help researc

# Evaluate

In [24]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
rag_eval_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=1
)

# We can save the dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset_rerank.json")

100%|██████████| 108/108 [05:45<00:00,  3.20s/it]


In [26]:
from llama_index.finetuning.embeddings.common import (
    EmbeddingQAFinetuneDataset,
)
rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "./rag_eval_dataset_rerank.json"
)

In [27]:
import pandas as pd

def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [30]:
from llama_index.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(similarity_top_k=i, node_postprocessors=[cohere_rerank])
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    Retriever Name  Hit Rate      MRR
0  Retriever top_2  0.661308  0.54716
    Retriever Name  Hit Rate       MRR
0  Retriever top_4  0.773848  0.580743
    Retriever Name  Hit Rate       MRR
0  Retriever top_6  0.826367  0.590014
    Retriever Name  Hit Rate       MRR
0  Retriever top_8  0.856377  0.595979
     Retriever Name  Hit Rate       MRR
0  Retriever top_10  0.871383  0.596152


It's important to keep in mind that all the results above are based on only two samples even when the retriever fetch 10 items from the vector store. So, it means that instead of passing 10 chunks of data which translates into more API usage and higher cost, we will get the same quality by passing 2 chunk of data.